**Muhammad Mohsin**

**i170216@nu.edu.pk**

MOUNTING DRIVE


In [98]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Libraries

In [99]:
import numpy as np
from collections import defaultdict 
import re

READING FILE

In [100]:
#=====================================================================FILE READING===============================================================
def readFile(file_loc): 
  file1 = open(file_loc, 'r')
  Lines = file1.readlines()
  
  rawData = []
  count = 0
  # Strips the newline character
  for line in Lines: 
    line = line.strip() 
    if line == "\n" or line == '': 
      pass
    else:
      rawData.append(line)  

  return rawData 



TOKENZING THE DATA

In [101]:
#===================================================================== TOKENIZING ===============================================================
#FOR CHARACHTER BIGRAM & UNIGRAM MODEL 
def char_tokenize(rawData): 
  tokenz = []
  for line in rawData: 
    token = line.split() 
    for word in token:  
      tokenz.append(word)
  return tokenz

 
  

CREATING CHARACTER LEVEL BIGRAM

In [102]:
def generate_character_bigram(char_tokenz): 
  bigram = []
  c_bigram = [] 
  ngram = defaultdict(int)
  for i in range(0, len(char_tokenz)): 
    for j in range(0, len(char_tokenz[i])-1): 
      ngram[char_tokenz[i][j] + char_tokenz[i][j+1]] +=1
  
  for w,c in ngram.items(): 
    bigram.append(w) 
    c_bigram.append(c) 
    
  return bigram,c_bigram 
 

CREATING UNIGRAMS

In [103]:
def generate_unigram(ngram_token):
  unigram = defaultdict(int)
  for word in ngram_token:
    unigram[word] += 1 # increment element's value by 1
  return unigram

READING MISSPELLING.TXT

In [104]:
#=====================================================================FILE READING===============================================================
def read_Mispelled(file_loc): 
  file1 = open(file_loc, 'r')
  Lines = file1.readlines()
  
  rawData = []
  count = 0
  # Strips the newline character
  for line in Lines: 
    line = line.strip()  
    if count == 0:  
      count+=1; 
      pass 
    else:
      count+=1 
      if line == "\n" or line == '': 
        pass
      else:
        rawData.append(line)   

  for i in range(len(rawData)):    
    rawData[i] = re.split(' |,|\*|\t',rawData[i]) 
    while '' in rawData[i]:
      rawData[i].remove('')
    
  
  return rawData 





Error Model Edit Tables

In [105]:
def createTable(misspelled): 
  #OPERATIONS 
  #1 : Insert
  #2 : Delete 
  #3 : Subsiture 
  #4 : Traversal 

  delete = np.zeros((27, 26)) 
  insert = np.zeros((27, 26)) 
  subsitute = np.zeros((26, 26)) 
  traversal = np.zeros((26, 26))
  
  for i in range (0, len(misspelled)): 
    correct = misspelled[i][0]  
    
    for j in range(1, len(misspelled[i])):   
      wrong  = misspelled[i][j] 
      if len(wrong)> len(correct): # YANI INSERT HO RHA HAI   
        temp_corrected = correct # cz change karna hai
        temp_corrected+='.' 
        for k in range(0,len(wrong)):        #loop to iterate the word   
          if(wrong[k] != temp_corrected[k]):  
            if k!= 0:  
              idx_i = ord(correct[k-1]) - 97 
              idx_j = ord(wrong[k]) - 97 
              insert[idx_i][idx_j] += 1
              break
            else: 
              idx_j = ord(wrong[k]) - 97 
              insert[26][idx_j] +=1 
              break


      elif len(correct)> len(wrong): #YANI DELETION HO RHI HAI  
        temp_wrong = wrong # cz change karna hai
        temp_wrong+= '.' 
        for k in range(len(correct)):       #loop to iterate the word  
          if(temp_wrong[k] != correct[k]):  
            if k!= 0:  
              idx_i = ord(correct[k-1]) - 97 
              idx_j = ord(correct[k]) - 97 
              delete[idx_i][idx_j] += 1 
              break
            else: 
              idx_j = ord(correct[k]) - 97  
              delete[26][idx_j] +=1 
              break
      
       
      elif len(correct) == len(wrong): #YANI SUBSITUTION ya TRAVERSAL HO RHa HAI 
        if (wrong[len(wrong)-1] != correct[len(correct)-1] and  wrong[len(wrong)-2] == correct[len(correct)-2] ): 
          idx_i = ord(wrong[len(wrong)-1]) - 97 
          idx_j = ord(correct[len(correct)-1]) - 97  
          subsitute[idx_i][idx_j] += 1
        else: 
          for k in range(len(correct)-1):       #loop to iterate the word  
            if wrong[k] != correct[k]: 
              if wrong[k+1] == correct[k] and wrong[k] == correct[k+1]: #to check traversal  
                idx_i = ord(correct[k]) - 97 
                idx_j = ord(correct[k+1]) - 97  
                traversal[idx_i][idx_j] += 1 
                break
              else:  #warna subsiture hoga
                idx_i = ord(wrong[k]) - 97 
                idx_j = ord(correct[k]) - 97  
                subsitute[idx_i][idx_j] += 1
                break  

  return insert, delete, subsitute, traversal
  



Finding **P(X|W)**

In [106]:
def find_Prob_X_given_W(insert, delete, subsitute, traversal, correct, wrong, bigram, bigram_count):
  #INSERTION 
  if len(wrong)> len(correct): # YANI INSERT HO RHA HAI   
    temp_corrected = correct # cz change karna hai
    temp_corrected+='.' 
    for k in range(0,len(wrong)):        #loop to iterate the word   
      if(wrong[k] != temp_corrected[k]):  
        if k!= 0:  
          idx_i = ord(correct[k-1]) - 97 
          idx_j = ord(wrong[k]) - 97 
          row_sum = np.sum(insert, axis = 1)   
          denominator = row_sum[idx_i]
          numerator = insert[idx_i][idx_j]
          return numerator/ denominator  
        else: 
          idx_i = 26
          idx_j = ord(wrong[k]) - 97 
          insert[idx_i][idx_j] +=1 
          row_sum = np.sum(insert, axis = 1)   
          denominator = row_sum[idx_i]
          numerator = insert[idx_i][idx_j]
          return numerator/ denominator  
  
  #DELETION
  elif len(correct)> len(wrong): #YANI DELETION HO RHI HAI  
      temp_wrong = wrong # cz change karna hai
      temp_wrong+= '.' 
      for k in range(len(correct)):       #loop to iterate the word  
        if(temp_wrong[k] != correct[k]):  
          if k!= 0:  
            idx_i = ord(correct[k-1]) - 97 
            idx_j = ord(correct[k]) - 97 
            bigram_char = correct[k-1] + correct[k] 
             
            denominator = count_bigram[bigram.index(bigram_char)] 
            if denominator == 0:  
              denominator = 100000
            numerator = delete[idx_i][idx_j]
            return numerator/ denominator
          
          else: 
            idx_i = 26 
            idx_j = ord(correct[k]) - 97  
            bigram_char = correct[k-1] + correct[k] 
            denominator = count_bigram[bigram.index(bigram_char)] 
            if denominator == 0: 
              denominator = 100000
            numerator = delete[idx_i][idx_j]
            return numerator/ denominator   
  
  #TRAVERSAL AND SUBSITUTION 
  elif len(correct) == len(wrong): #YANI SUBSITUTION ya TRAVERSAL HO RHa HAI 
    if (wrong[len(wrong)-1] != correct[len(correct)-1] and  wrong[len(wrong)-2] == correct[len(correct)-2] ): 
      idx_i = ord(wrong[len(wrong)-1]) - 97 
      idx_j = ord(correct[len(correct)-1]) - 97  
      col_sum = np.sum(insert, axis = 0)   
      denominator = col_sum[idx_j]
      numerator = subsitute[idx_i][idx_j] 
      return numerator/ denominator
    else:
      for k in range(len(correct)-1):       #loop to iterate the word  
        if wrong[k] != correct[k]: 
          if wrong[k+1] == correct[k] and wrong[k] == correct[k+1]: #to check traversal  
            idx_i = ord(correct[k]) - 97 
            idx_j = ord(correct[k+1]) - 97  
            bigram_char = correct[k] + correct[k+1]    
            denominator = count_bigram[bigram.index(bigram_char)] 
            if denominator == 0: 
              denominator = 10000
            numerator = traversal[idx_i][idx_j]
            return numerator/ denominator 
          else:  #warna subsiture hoga
            idx_i = ord(wrong[k]) - 97 
            idx_j = ord(correct[k]) - 97  
            col_sum = np.sum(insert, axis = 0)   
            denominator = col_sum[idx_j]
            numerator = subsitute[idx_i][idx_j]
            return numerator/ denominator  
          

Check the word has traversal or not

e.g

correct: cares

wrong: acres

In [107]:
def hasTraversal(wrong, correct): 
  wrong_unigram = []  
  wrong_unigram_count = [] 
  correct_unigram = []  
  correct_unigram_count = []  

  for i in range(0, len(wrong)):
    if wrong[i] not in wrong_unigram:
      wrong_unigram.append(wrong[i])
      wrong_unigram_count.append(1)
    else:
      wrong_unigram_count[wrong_unigram.index(wrong[i])] +=1 
      
    if correct[i] not in correct_unigram:
      correct_unigram.append(correct[i])
      correct_unigram_count.append(1)
    else:
      correct_unigram_count[correct_unigram.index(correct[i])] +=1  
  
  traversal = True 
  if len(correct_unigram) != len(wrong_unigram): 
    return False
  else:  
    for i in range(len(correct_unigram)): 
      if correct_unigram[i] not in wrong_unigram: 
        traversal = False 
        return False
      elif wrong_unigram_count[wrong_unigram.index(correct_unigram[i])] != correct_unigram_count[i] : 
        traversal = False 
        return False 

  for k in range(len(wrong)-1): 
    if wrong[k] != correct[k]: 
      if wrong[k+1] == correct[k] and wrong[k] == correct[k+1]: #to check traversal    
          return True
  
  return False


Generating Candidate

In [108]:
import nltk
def generate_candidates(misspell, unigram): 
  candidates = []
  for word,_ in unigram.items():
    edit_distance = nltk.edit_distance(misspell,word, substitution_cost=1)    
    if edit_distance == 1: 
      candidates.append(word) 
    elif edit_distance == 2: 
      if len(word) == len(misspell): 
        if hasTraversal(word, misspell)== True: 
          candidates.append(word)

  return candidates 




Finding the most Probable word

In [134]:
def get_most_probable(candidates, misspell, insert, delete, subsitute, traversal, bigram, count_bigram, unigram):    
  probabilities = []
  for word in candidates:   
    P_X_W = find_Prob_X_given_W(insert, delete, subsitute, traversal, word, misspell, bigram, count_bigram)   
    P_W  = 0 
    for w,c in unigram.items(): 
      if w == word: 
        P_W = c
        break 
    prob = P_X_W * P_W  
    probabilities.append(prob)

  correct_word = candidates[np.argmax(probabilities)] 
  print("Word:", misspell)  
  print("Candidate Word with probability")
  for i in range(len(candidates)): 
    print(candidates[i], probabilities[i])
  print("The most probable word is", correct_word)  
  print("\n")
  return correct_word



Check the word is misspelled

In [110]:
def isMisspelled(unigram, misspell): 
  for word in unigram: 
    if word == misspell: 
      return False
  return True

DRIVER CODE

In [111]:
rawData = readFile("/content/drive/MyDrive/NLP4/data.txt")  
char_tokenz = char_tokenize(rawData) 

In [112]:
unigram = generate_unigram(char_tokenz)
bigram,count_bigram = generate_character_bigram(char_tokenz) 

In [113]:
import numpy as np 
misspelled = read_Mispelled("/content/drive/MyDrive/NLP4/misspellings.txt") 
insert, delete, subsitute, traversal = createTable(misspelled) 


EXAMPLE 1

In [123]:
wrong_word = "terhuan"

if isMisspelled(unigram, wrong_word): 
  candidates = generate_candidates(wrong_word, unigram)
  get_most_probable(candidates, wrong_word, insert, delete, subsitute, traversal, bigram, count_bigram, unigram)
else: 
  print("Word is correctly spelled")

Word: terhuan
Candidate Word with probability
terhan 15540.59315995725
The most probable word is terhan




EXAMPLE 2

In [124]:
wrong_word = "satjh"

if isMisspelled(unigram, wrong_word): 
  candidates = generate_candidates(wrong_word, unigram)
  get_most_probable(candidates, wrong_word, insert, delete, subsitute, traversal, bigram, count_bigram, unigram)
else: 
  print("word is correctly spelled")

Word: satjh
Candidate Word with probability
sath 14108.613414634146
sajh 2.196432934278036
samjh 20.67153951576062
satah 1360.3228481555618
sateh 3.4571428571428573
The most probable word is sath




EXAMPLE 3

In [135]:
wrong_word = "hiz"

if isMisspelled(unigram, wrong_word): 
  candidates = generate_candidates(wrong_word, unigram)
  get_most_probable(candidates, wrong_word, insert, delete, subsitute, traversal, bigram, count_bigram, unigram)
else: 
  print("word is correctly spelled")

Word: hiz
Candidate Word with probability
hi 6931.7816858702245
hit 17.162912022765
hia 27.932513583071202
him 23.357240749200546
iz 8.596437380104138
his 72.22961890979258
chiz 687.6
hii 73.30705079605762
hil 26.666291629162917
hin 26.162162162162165
aiz 205.6188161281098
hik 10.650140318054257
ziz 22.333560245064668
hir 80.89259075087021
hip 6.647703427372348
hiv 0.6585923217550275
hie 3.7142857142857144
uiz 0.585216572504708
hid 0.36954481464101363
haz 4.323706033743209
hez 0.7310344827586206
biz 0.3360258481421648
hifz 0.004052296805540156
fiz 1.3468354430379748
hz 0.20520128250801567
hoz 2.6984888462461023
haiz 0.0015644477917116
hizo 0.0
diz 0.14500234631628342
hij 0.17946990116801437
The most probable word is hi




EXAMPLE 4

In [127]:
wrong_word = "lekiy"

if isMisspelled(unigram, wrong_word): 
  candidates = generate_candidates(wrong_word, unigram)
  get_most_probable(candidates, wrong_word, insert, delete, subsitute, traversal, bigram, count_bigram, unigram)
else: 
  print("word is correctly spelled")

Word: lekiy
Candidate Word with probability
lekin 15705.539539539539
The most probable word is lekin




Complete Sentence as an Input

In [143]:
sentence = "yaar xyeh saii hai lekiq" 
token = sentence.split() 
new_sent = [] 
for i in range(len(token)): 
  if isMisspelled(unigram, token[i]): 
    candidates = generate_candidates(token[i], unigram)
    word = get_most_probable(candidates, token[i], insert, delete, subsitute, traversal, bigram, count_bigram, unigram) 
    new_sent.append(word)
  else: 
    new_sent.append(token[i]) 

separator = ' '
print("Misspelled Sentence:", sentence)
print("Corrected Sentence:", separator.join(new_sent))




Word: xyeh
Candidate Word with probability
yeh 31762.19257311863
The most probable word is yeh


Word: saii
Candidate Word with probability
sai 58.63250454821104
sahi 248.77420153503343
saif 0.16570771001150747
saki 18.650608044901777
sahii 0.00012187176983387824
sari 37.08876181004475
aii 9.47779177391971
said 18.548568747067105
haii 18.92151522654122
sali 1.6255625562556255
sami 0.46870717222476016
saib 14.43987999076852
sadi 51.80807132801502
sani 21.47172172172172
sii 4.19818260493293
sagi 0.14560439560439561
naii 2.403403403403403
sabi 0.30994691899376875
gaii 0.5393772893772893
safi 0.2623705408515535
sain 1.1961961961961962
savi 0.02056672760511883
sair 189.19542516161113
saini 0.008444502036956207
saji 0.35130278526504943
saqi 1.2525252525252526
sati 0.23049561299502014
sais 0.15903614457831325
saiu 0.06779661016949153
The most probable word is sahi


Word: lekiq
Candidate Word with probability
lekin 15916.824824824826
The most probable word is lekin


Misspelled Sentence: yaar